**Task**: Run error correction for 7-qb Steane code
* Build the Stean EC circuit.
* Run the circuit on SparseSim to track propagation of stabilizer group through circuit.
* Add noise, measure the syndrome and perform necessary correction.

In [1]:
import pecos as pc
import numpy as np

**(a)** Run circuit, apply correction if necessary to produce a $|0_L\rangle$ state on the data qubits.
* The $|0_L\rangle$ state is produced by inputting $|0000000\rangle$ on the 7 data qubits, run the X-plaquette circuit and apply corrections for measured syndromes.
    * Why do we only need to run the X-plaquette circuit? $\rightarrow$ because the $|0000000\rangle$ state is already a +1 eigenstate of the $Z_L$ operator, thus, as all the Z-stabilizer (generators) commute with $Z_L$, $|0000000\rangle$ will pass the check with syndrome $|000\rangle$ (i.e. no error) and the state remains unchanged.
    * For $X_L$, however, $|0000000\rangle$ is not a +1 eigenstate. The X-plaquette circuit will entangle the state as follows:
    * The first plaquette group takes the state to $\frac{1}{2}(|0000000\rangle+|0001111\rangle)|0\rangle_7+\frac{1}{2}(|0000000\rangle-|0001111\rangle)|1\rangle_7$. This is after applying a Hadamard gate, s.t. we can measure the state in the Z-basis. The outcome will be 50% chance of either $|0\rangle_7$ or $|1\rangle_7$. Only for the latter we see there is a phase error somewhere in this group, which we can identify with the help of the other plaquettes and subsequently correct.
    * After all 3 X-plaquette measurements we collapsed the state to one of $2^3=8$ possible superposition states which can be corrected by applying a single Z-gate to one of the 7 qubits. (The 8th possibility is the error free state $|0_L\rangle=|0000000\rangle+|0001111\rangle+|1010101\rangle+|0110011\rangle+|1011010\rangle+|1100110\rangle+|0111100\rangle+|1101001\rangle$, where no correction is needed)
* It is striking that the superposition state $|0_L\rangle$ corresponds to the (normalized) linear combination of the bitstrings of all Z-(and also X)-stabilizers: $\mathcal S=\{IIIIIII,IIIZZZZ,ZIZIZIZ,IZZIIZZ,ZIZZIZI,ZZIIZZI,IZZZZII,ZZIZIIZ\}$ 

In [102]:
n_qbs = 10 # 7 data_qbs, 3 msmt_qbs

k1 = 0b0001111
k2 = 0b1010101
k3 = 0b0110011
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3]

plaquettes = [(3,4,5,6),(0,2,4,6),(1,2,5,6)] # correspond to stab set.
msmt_qbs = [7,8,9]
COR_TABLE = { 7: 3, 8: 0, 9: 1, 7+8: 4, 8+9: 2, 7+9: 5, 7+8+9: 6 }

# Create bitstring from list of indices for '1's left to right (MSB) 
def bin_from_ids(indices, offset=0, bitlen=7):
    bit_string = 0
    for index in indices:
        bit_string |= 2**(bitlen-index-1+offset) # 7-bit number 2**0..6
    return bit_string

# Steane plaquette measurment circuits
steane_x = pc.circuits.QuantumCircuit()
steane_x.append('init |0>', {qb for qb in range(n_qbs)}) # Just to be explicit, would also automatically be initialized to |0>.
steane_x.append('H', {7,8,9}) # msmt qubits to |+>
for m_qb, plaquette in zip(msmt_qbs, plaquettes):
    for d_qb in plaquette:
        steane_x.append('CNOT', {(m_qb, d_qb)}) # detect and correct Z errors
steane_x.append('H', {7,8,9}) # msmt qubits to |0>
steane_x.append('measure Z', {7,8,9}) # measure msmt qbs in Z-basis

# Run plaquette msmts
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
state = pc.simulators.SparseSim(n_qbs)
x_msmt, _ = circ_runner.run(state, steane_x) # Produce superposition state, measure syndrome

# Apply corrective action
x_syn = sum(list(*x_msmt.values())) # apply opposite correction
if x_syn: 
    state.run_gate('Z', {COR_TABLE[x_err_id]}) # correct by opposite gate!
    print('X Syndrome', *x.values(), 'caused Z correction on qb.', COR_TABLE[x_err_id])

# Verify that the constructed state is |0_L>
qc = pc.circuits.QuantumCircuit()
qc.append('measure Z', {0,1,2,3,4,5,6})
msmt, _ = circ_runner.run(state, qc) # collapses superposition

# Check if measured state is in superposition of Steane |0_L> state.
hamming2 = lambda a,b: bin(a^b).count('1')
msmt_str = bin_from_ids(list(*msmt.values()))
if min([hamming2(msmt_str, stab) for stab in stabs]) == 0:
    print("Measured",bin(msmt_str),"which is one of the states in the |0_L> superposition.")

X Syndrome {8: 1, 7: 1} caused Z correction on qb. 4
Measured 0b1111 which is one of the states in the |0_L> superposition.


**(b)** Run the same circuit in the face of noise. This time, repeat the circuit until we measure a logical error, i.e. a state which is in the $|1_L\rangle$ superposition.

In [151]:
# Create the noise model
class DepolarGen(pc.error_gens.parent_class_error_gen.ParentErrorGen):
    
    two_qubit_gates = {'CNOT', 'CZ', 'SWAP', 'G', 'MS', 'SqrtXX', 'RXX'}
    one_qubit_gates = {'I', 'X', 'Y', 'Z', 'Q', 'Qd', 'R', 'Rd', 'S', 'Sd', 
                       'H', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H+z+x', 'H-z-x', 
                       'H+y-z', 'H-y-z', 'H-x+y', 'H-x-y', 'F1', 'F1d', 'F2', 
                       'F2d', 'F3', 'F3d', 'F4', 'F4d', 'RX', 'RY', 'RZ'}
    error_one_paulis_collection = ['X','Y','Z']
    error_two_paulis_collection = [
            ('I', 'X'), ('I', 'Y'), ('I', 'Z'),
            ('X', 'I'), ('X', 'X'), ('X', 'Y'), ('X', 'Z'),
            ('Y', 'I'), ('Y', 'X'), ('Y', 'Y'), ('Y', 'Z'),
            ('Z', 'I'), ('Z', 'X'), ('Z', 'Y'), ('Z', 'Z')
    ]
    
    def start(self, circuit, error_params, state):
        super().start(circuit, error_params)
        
    def generate_tick_errors(self, tick_circ, time, **params):
        if type(time) == tuple: tick_index = time[-1]
        else: tick_index = time
        
        before = pc.circuits.QuantumCircuit() # faults before tick
        after  = pc.circuits.QuantumCircuit() # faults after tick
        
        q0 = lambda locs, fault, r: [{fault: {l}} for l in locs if np.random.random() <= r]
        q1 = lambda locs, faults, r: [{np.random.choice(faults): {l}} for l in locs if np.random.random() <= r]
        q2 = lambda locs, faults, r: [{f: {l}} for ltup in locs  if np.random.random() <= r for (l,f) in zip(ltup, faults[np.random.choice(len(faults))])]
        
        qc_append = lambda qc, circuit_setup: [qc.append(tick) for tick in circuit_setup]
        
        for sym, locs, _ in tick_circ.circuit.items(tick_index):
            if sym in {'init |0>', 'init |1>'}:
                qc_append(after, q0(locs, 'X', self.error_params['r']))
            elif sym in {'init |+>', 'init |->', 'init |+i>', 'init |-i>'}:
                qc_append(after, q0(locs, 'Z', self.error_params['r']))
            elif sym in {'measure Z'}:
                qc_append(before, q0(locs, 'X', self.error_params['q']))
            elif sym in {'measure X', 'measure Y'}:
                qc_append(before, q0(locs, 'Z', self.error_params['q']))
            elif sym in self.one_qubit_gates:
                qc_append(after, q1(locs, self.error_one_paulis_collection, self.error_params['p1']))
            elif sym in self.two_qubit_gates:
                qc_append(after, q2(locs, self.error_two_paulis_collection, self.error_params['p2']))
            elif symbol == 'wait_initial' or symbol == 'wait_final':
                pass
            else:
                raise Exception("This error model doesn't handle gate: %s!" % symbol)
        self.error_circuits.add_circuits(time, before_faults=before, after_faults=after)
        return self.error_circuits

# Steane Plaquette circuits for given basis
def plaquette_qc(basis):
    qc = pc.circuits.QuantumCircuit()
    qc.append('init %s' % ('|0>' if basis == 'Z' else '|+>'), {7,8,9})
    for m_qb, plaquette in zip(msmt_qbs, plaquettes):
        for d_qb in plaquette:
            if basis == 'Z': qc.append('CNOT', {(d_qb, m_qb)})   # detect and correct X errors
            elif basis == 'X': qc.append('CNOT', {(m_qb, d_qb)}) # detect and correct Z errors
    qc.append('measure %s' % basis, {7,8,9}) # measure msmt qbs
    return qc

# Run circuit with error model
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
measure_z = pc.circuits.QuantumCircuit([{'measure Z': {0,1,2,3,4,5,6}}])

k1 = 0b0001111
k2 = 0b1010101
k3 = 0b0110011
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3]
hamming2 = lambda a,b: bin(a^b).count('1')

min_dist = 2
count = 0
p = 0.01

while min_dist >= 1: # break if one run has min_dist=0
    
    # Run Plaquette measurements
    state = pc.simulators.SparseSim(n_qbs)
    e_params = {'q':p,'p1':p,'p2':p,'r':p}
    x, e_x = circ_runner.run(state, steane_x, error_gen=DepolarGen(), error_params=e_params)
    z, e_z = circ_runner.run(state, steane_z, error_gen=DepolarGen(), error_params=e_params)

    # Apply corrective action, if any
    x_err_id = sum(list(*x.values()))
    z_err_id = sum(list(*z.values()))
    if x_err_id: state.run_gate('Z', {COR_TABLE[x_err_id]})
    if z_err_id: state.run_gate('X', {COR_TABLE[z_err_id]})

    # Measure data qubits: collapse to one state in superposition
    msmt, _ = circ_runner.run(state, measure_z)
    msmt_str = bin_from_ids(list(*msmt.values()))
    
    # Record minimum distance of X_L|msmt> to each stab
    # If distance = 0: Measured part of superposition of |1_L>
    min_dist = min([hamming2(msmt_str^0b1111111, stab) for stab in stabs])
    count += 1 # increase counter

err2ticks = lambda errs: [list(err.values())[0]._ticks for _, err in errs.items()]
print("Correction to",bin(msmt_str),"(|1_L>) after %d runs" % count,"due to errors:")
print("X-circuit:", err2ticks(e_x))
print("Z-circuit:", err2ticks(e_z))

Correction to 0b1110000 (|1_L>) after 41 runs due to errors:
X-circuit: [[Tick({'Y': {7}}), Tick({'I': {4}})]]
Z-circuit: []
